In [1]:
!mkdir myhaproxy

In [2]:
!cd myhaproxy/

In [3]:
!pwd

/home/agungw132/Dropbox/Jupyter/influxdb-cluster


In [4]:
ls

6 - Setting HAProxy & LB.ipynb
7- Trial.ipynb
Install telegraf (to store data in influxdb) .ipynb
myhaproxy/
Step 1 - Installing an InfluxDB Enterprise cluster.ipynb
Step 2 - Set up the metanodes.ipynb
Step 3 - Set up the data nodes.ipynb
Step 4 - Join the nodes to the cluster.ipynb
telegraf.conf


In [5]:
!docker pull haproxy

Using default tag: latest
latest: Pulling from library/haproxy

81be8156: Already exists 
22a0d424: Pulling fs layer 
Digest: sha256:6850da852c4e2bd8b613a6cc7e77946c5ee9c66887b693f87a8a2c0598b3d62b
Status: Downloaded newer image for haproxy:latest


In [6]:
!docker images

REPOSITORY            TAG                 IMAGE ID            CREATED             SIZE
haproxy               latest              991c6e578a0d        3 hours ago         69.5MB
awesome               v2                  02e56bf30362        28 hours ago        674MB
awesome               latest              50381a421f3e        29 hours ago        674MB
ubuntu                16.04               7aa3602ab41e        4 days ago          115MB
haproxy               <none>              3b10abf01813        6 days ago          69.5MB
node                  latest              52fe93b8eea7        11 days ago         674MB
redis                 latest              f06a5773f01e        2 weeks ago         83.4MB
dockercloud/haproxy   <none>              4d6ae6c16c4d        7 months ago        42.6MB


In [8]:
%%file myhaproxy/haproxy.cfg
defaults
log     global
mode    tcp
option  tcplog

frontend mqtt
    bind 0.0.0.0:8086
    default_backend mqtt

backend mqtt
    mode tcp
    balance roundrobin
    server influx-cluster-1 172.17.0.2:8086 check
    server influx-cluster-2 172.17.0.3:8086 check
    server influx-cluster-3 172.17.0.4:8086 check


Writing myhaproxy/haproxy.cfg


In [9]:
!docker run -d --name my-running-haproxy -v $PWD/myhaproxy:/usr/local/etc/haproxy:ro haproxy

8cd4a384151f5c7bb8af41f44e389fc739e03aba0414c6ed616658daa1a1b7ce


In [12]:
!docker ps

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS               NAMES
8cd4a384151f        haproxy             "/docker-entrypoint.…"   16 seconds ago      Up 16 seconds                           my-running-haproxy
9e04dcafd7a6        ubuntu:16.04        "sleep 9999999999999…"   21 hours ago        Up 21 hours                             influx-cluster-3
4a5974247f70        ubuntu:16.04        "sleep 9999999999999…"   21 hours ago        Up 21 hours                             influx-cluster-2
232ae55d8ab4        ubuntu:16.04        "sleep 9999999999999…"   21 hours ago        Up 21 hours                             influx-cluster-1


In [14]:
!docker inspect 8cd

[
    {
        "Id": "8cd4a384151f5c7bb8af41f44e389fc739e03aba0414c6ed616658daa1a1b7ce",
        "Created": "2018-07-31T19:23:02.132367798Z",
        "Path": "/docker-entrypoint.sh",
        "Args": [
            "haproxy",
            "-f",
            "/usr/local/etc/haproxy/haproxy.cfg"
        ],
        "State": {
            "Status": "running",
            "Running": true,
            "Paused": false,
            "Restarting": false,
            "OOMKilled": false,
            "Dead": false,
            "Pid": 9681,
            "ExitCode": 0,
            "Error": "",
            "StartedAt": "2018-07-31T19:23:02.464628891Z",
            "FinishedAt": "0001-01-01T00:00:00Z"
        },
        "Image": "sha256:991c6e578a0d6f0b1c2e0ddf6fcf0ef5a9de18283dc403c4479e5946dbe5408f",
        "ResolvConfPath": "/var/lib/docker/containers/8cd4a384151f5c7bb8af41f44e389fc739e03aba0414c6ed616658daa1a1b7ce/resolv.conf",
        "HostnamePath": "/var/lib/docker/containers/8cd4a384151f5c7bb8af41

In [16]:
#direct to one of the node

!influx -host 172.17.0.2 -execute "show databases"

name: databases
---------------
name
_internal
telegraf



In [17]:
#via haproxy

!influx -host 172.17.0.5 -execute "show databases"

name: databases
---------------
name
_internal
telegraf



In [20]:
!influx -host 172.17.0.2 -execute "create user admin with password 'admin' with all privileges"
!influx -host 172.17.0.2 -execute "auth admin admin"
!influx -host 172.17.0.2 -execute "show users"

user	admin
admin	true



In [23]:
!influx -host 172.17.0.3 -execute "create user admin with password 'admin' with all privileges"
!influx -host 172.17.0.3 -execute "auth admin admin"
!influx -host 172.17.0.3 -execute "show users"

ERR: unable to parse authentication credentials
Please set a database with the command "use <database>".
unable to parse authentication credentials
ERR: unable to parse authentication credentials
Please set a database with the command "use <database>".
unable to parse authentication credentials


In [24]:
#create 2 instance of HAproxy
!docker run -d --name my-running-haproxy-2 -v $PWD/myhaproxy:/usr/local/etc/haproxy:ro haproxy

57dc8116bf4ef30ad1768aedd63d9936005d580ccbb44b1c9441c9efc3512e6e


In [25]:
!docker inspect my-running-haproxy-2

[
    {
        "Id": "57dc8116bf4ef30ad1768aedd63d9936005d580ccbb44b1c9441c9efc3512e6e",
        "Created": "2018-08-01T01:37:39.217614429Z",
        "Path": "/docker-entrypoint.sh",
        "Args": [
            "haproxy",
            "-f",
            "/usr/local/etc/haproxy/haproxy.cfg"
        ],
        "State": {
            "Status": "running",
            "Running": true,
            "Paused": false,
            "Restarting": false,
            "OOMKilled": false,
            "Dead": false,
            "Pid": 11935,
            "ExitCode": 0,
            "Error": "",
            "StartedAt": "2018-08-01T01:37:39.542150849Z",
            "FinishedAt": "0001-01-01T00:00:00Z"
        },
        "Image": "sha256:991c6e578a0d6f0b1c2e0ddf6fcf0ef5a9de18283dc403c4479e5946dbe5408f",
        "ResolvConfPath": "/var/lib/docker/containers/57dc8116bf4ef30ad1768aedd63d9936005d580ccbb44b1c9441c9efc3512e6e/resolv.conf",
        "HostnamePath": "/var/lib/docker/containers/57dc8116bf4ef30ad1768